<a href="https://colab.research.google.com/github/sreehari59/Polar-Embedding/blob/main/Polar_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
from numpy import linalg
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

# This is for the plotly subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# This is to load file from google drive
from google.colab import drive
drive.mount('/content/drive')

# This is to download csv file
from google.colab import files

import random
import scipy


!pip install ipython-autotime
%load_ext autotime

In [ ]:
def unzipping_file():
  !wget https://nlp.stanford.edu/data/glove.6B.zip
  !unzip glove*.zip # When it is unzipped you can see the different set of dimension it can have that is 25d, 50d, 100d or 200d

time: 1.91 ms (started: 2022-05-08 14:03:16 +00:00)


In [ ]:
def model_initialization(unzippedname, new_name):
  model_glove = glove2word2vec(unzippedname,new_name)
  model_glove = gensim.models.KeyedVectors.load_word2vec_format(new_name, binary=False)
  return model_glove

time: 2.65 ms (started: 2022-05-08 16:18:53 +00:00)


In [ ]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))  
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    
    temp_file.close()
    return output_path

time: 8.27 ms (started: 2022-05-08 16:32:15 +00:00)


In [ ]:
def load_normalized_embedding(output_file_name):
  return gensim.models.KeyedVectors.load_word2vec_format(output_file_name,binary=True)

time: 4.89 ms (started: 2022-05-08 16:19:16 +00:00)


**Loading the Antonym**

In [ ]:
def load_antonyms(list_new,current_model):
  list_new= list(dict.fromkeys(list_new).keys())

  similarity_matrix = defaultdict(list)
  for each_pair in tqdm(list_new):
      word1 = each_pair[0]
      word2 = each_pair[1]
      if word1 < word2:
          similarity_matrix[word1].append(word2)
      else:
          similarity_matrix[word2].append(word1)

  all_similarity = defaultdict(dict)
  for each_key in tqdm(similarity_matrix):
      for each_value in similarity_matrix[each_key]:
  #         cosine_similarity([current_model[each_key]]
          try:
            all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
          except:
            print("The word is not in the vocabulary : ",each_key," ",each_value)

  final_list = []
  for index_counter, each_key in enumerate(tqdm(all_similarity)):
  #     print(each_key,all_similarity[each_key])
      listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
  #     print(listofTuples)
      final_list.append((each_key, listofTuples[0][0]))
  print(len(final_list))

  list_antonym = final_list
  
  return list_antonym

time: 27 ms (started: 2022-05-08 16:23:54 +00:00)


**Find the antonym difference vectors**

In [ ]:
def antonym_difference_vector(list_antonym,current_model):
  num_antonym  = len(list_antonym)
  antonymy_vector = []
  for each_word_pair in list_antonym:
      antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
      
  antonymy_vector = np.array(antonymy_vector)
  print(antonymy_vector.shape)
  print(antonymy_vector)
  return antonymy_vector

time: 5.59 ms (started: 2022-05-08 16:19:22 +00:00)


In [ ]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim,antonymy_vector):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

time: 4.3 ms (started: 2022-05-08 16:19:28 +00:00)


In [ ]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    #print('New model size is',len(current_model), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

time: 27.2 ms (started: 2022-05-08 16:19:31 +00:00)


In [ ]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict

time: 7.05 ms (started: 2022-05-08 16:19:58 +00:00)


In [ ]:
def download_csv_file(df,filename):
  df.to_csv(filename)
  files.download(filename)

time: 1.75 ms (started: 2022-05-08 16:20:03 +00:00)


In [ ]:
unzipping_file()
model_glove = model_initialization('glove.6B.300d.txt','gensim_glove.6B.300d.txt')
path = generate_norm_embedding(model_glove,'glove_norm_wiki.mod')
current_model  = load_normalized_embedding(path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/400000 [00:00<?, ?it/s]

time: 1min 22s (started: 2022-05-08 16:32:27 +00:00)


In [ ]:
list_new= [('product', 'service'),
('essential', 'luxury'),
('technical', 'natural'),
('renewable', 'nonrenewable'),
('advertising', 'rumour'), 
('lease', 'sell'), 
('tangible', 'intangible'), 
('demand', 'supply'),
('child', 'childless'),
('remote', 'physical')]

time: 6.61 ms (started: 2022-05-08 16:33:57 +00:00)


In [ ]:
list_antonym  = load_antonyms(list_new,current_model)

In [ ]:
antonymy_vector = antonym_difference_vector(list_antonym,current_model)
print(antonymy_vector)

In [ ]:
company = pd.read_csv('/content/drive/MyDrive/International_Fortune_GloVe.csv',encoding= 'unicode_escape')
name_list = company['0']

time: 8.3 ms (started: 2022-05-08 16:34:08 +00:00)


In [ ]:
name_word_embedding = dict()
new_name_list=[]
for name in name_list:
  try:
    name_word_embedding[name] = current_model[name]
    new_name_list.append(name)
  except:
    print("The company is not there: ",name)

print(new_name_list)

In [ ]:
random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))
name_new_embedding = generate_embedding_path(name_word_embedding, 'test_run',True ,random_antonym_vector,500,antonymy_vector)

10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/142 [00:00<?, ?it/s]

time: 85.3 ms (started: 2022-05-08 16:34:18 +00:00)


In [ ]:
antonym_500 = [list_antonym[x] for x in random_antonym_vector[:500]]

time: 1.65 ms (started: 2022-05-08 16:34:20 +00:00)


In [ ]:
df = dict()
for t in new_name_list:
  df[t] = make_polar_dict(t, antonym_500, name_new_embedding)

new_df = pd.DataFrame(df).transpose()


# change columns to better read names
new_columns = []

for pair in antonym_500:
  temp = pair[0]+''+pair[1]
  new_columns.append(temp)

new_df.columns = new_columns

time: 25.8 ms (started: 2022-05-08 16:34:23 +00:00)


In [ ]:
new_df.head()

,childchildless,advertisingrumour,naturaltechnical,nonrenewablerenewable,leasesell,essentialluxury,intangibletangible,demandsupply,productservice,physicalremote
walmart,-0.234927,0.053615,0.547627,0.094820,-0.203712,-0.397833,-0.107264,-0.610311,0.210923,-0.069312
amazon,0.256857,0.274870,0.337541,0.017428,-0.074484,0.149086,-0.073648,0.244374,0.154932,-0.792030
apple,0.059273,0.276218,-0.241936,-0.369896,-0.630734,0.019489,-0.029875,0.053345,0.446798,-0.351087
volkswagen,-0.164525,0.166854,-0.189240,-0.426518,-0.273070,-0.680670,-0.164260,-0.171160,0.358346,0.069241
mckesson,-0.725982,0.185271,0.196849,-0.052383,-0.190578,-0.033068,0.277886,-0.436756,-0.037312,0.300580


time: 18.9 ms (started: 2022-05-08 16:34:27 +00:00)


In [ ]:
input  = man - woman  